## This Notebook is for training different ML models on different kind of vectorization 

In [28]:
import pickle
import warnings
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from keras import backend as K
from sklearn.utils import shuffle
from keras.models import Sequential
from sklearn.pipeline import Pipeline
from molearn.classifiers.BR import BR
from keras.layers import Dense, Dropout
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from molearn.classifiers.Ensemble import Ensemble
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
from sklearn.metrics import classification_report,f1_score
from molearn.classifiers.classifier_chains import CC,RCC,MCC

In [2]:
warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = "all"

### Define some useful methods

In [3]:
def Hamming_loss(Ytest,Ypred):
    ''' Hamming loss aka Hamming distance '''
    return 1.-Hamming_score(Ytest,Ypred)

def Hamming_score(Ytest,Ypred):
    ''' Hamming score aka Hamming match '''
    N_test,L = Ytest.shape
    return np.sum((Ytest == Ypred) * 1.) / N_test / L

def Hamming_matches(Ytest,Ypred):
    N_test,L = Ytest.shape
    return np.sum((Ytest == Ypred) * 1.,axis=0) / N_test 

def Hamming_losses(Ytest,Ypred):
    return 1.-Hamming_matches(Ytest,Ypred)

def Exact_match(Ytest,Ypred):
    N_test,L = Ytest.shape
    return np.sum(np.sum((Ytest == Ypred) * 1,axis=1)==L) * 1. / N_test

In [4]:
def grid_search(train_x, train_y, test_x, test_y, parameters, pipeline):
    grid_search_tune = GridSearchCV(pipeline, parameters, cv=2, n_jobs=3, verbose=10)
    grid_search_tune.fit(train_x, train_y)

    print
    print("Best parameters set:")
    print (grid_search_tune.best_estimator_.steps)
    print

    # measuring performance on test set
    print ("Applying best classifier on test data:")
    best_clf = grid_search_tune.best_estimator_
    
    predictions = best_clf.predict(test_x)
    print('grid_search_tune.best_estimator : ',grid_search_tune.best_estimator_.steps[0])
    return best_clf

In [5]:
## save model
def save_model(clf,filename,folder):
    pickle.dump(clf, open("/home/sina/trained-model/"+folder+"/"+filename, 'wb'))

In [14]:
def bp_mll_loss(y_true, y_pred):
 
    # get true and false labels
    y_i = K.equal(y_true, K.ones_like(y_true))
    y_i_bar = K.not_equal(y_true, K.ones_like(y_true))
    
    # cast to float as keras backend has no logical and
    y_i = K.cast(y_i, dtype='float32')
    y_i_bar = K.cast(y_i_bar, dtype='float32')

    # get indices to check
    truth_matrix = pairwise_and(y_i, y_i_bar)

    # calculate all exp'd differences
    sub_matrix = pairwise_sub(y_pred, y_pred)
    exp_matrix = K.exp(-sub_matrix)

    # check which differences to consider and sum them
    sparse_matrix = exp_matrix * truth_matrix
    sums = K.sum(sparse_matrix, axis=[1,2])

    # get normalizing terms and apply them
    y_i_sizes = K.sum(y_i, axis=1)
    y_i_bar_sizes = K.sum(y_i_bar, axis=1)
    normalizers = y_i_sizes * y_i_bar_sizes
    results = sums / normalizers

    # sum over samples
    return K.sum(results)

In [15]:

# compute pairwise differences between elements of the tensors a and b
def pairwise_sub(a, b):
    column = K.expand_dims(a, 2)
    row = K.expand_dims(b, 1)
    return column - row

In [16]:
# compute pairwise logical and between elements of the tensors a and b
def pairwise_and(a, b):
    column = K.expand_dims(a, 2)
    row = K.expand_dims(b, 1)
    return K.minimum(column, row)

## classification methods

In [6]:
def logisticRegression(x_train, x_test, y_train, y_test):
        
    print ("LogisticRegression")
    pipeline = Pipeline([
        ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
    ])
    parameters = {
        "clf__estimator__C": [0.1,1,10],
        "clf__estimator__class_weight": [None],
    }
    clf = grid_search(x_train, y_train, x_test, y_test, parameters, pipeline)
    return clf

In [7]:
def adaboost(x_train, x_test, y_train, y_test):
        
    print ("LogisticRegression")
    pipeline = Pipeline([
        ('clf', OneVsRestClassifier(AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2),
    algorithm="SAMME"), n_jobs=-1)),
    ])
    parameters = {
        "clf__estimator__learning_rate": [1,1.5],
        "clf__estimator__n_estimators": [6],
    }
    clf = grid_search(x_train, y_train, x_test, y_test, parameters, pipeline)
    return clf

In [8]:
def naivebayes(x_train, x_test, y_train, y_test):
        
    print ("LogisticRegression")
    pipeline = Pipeline([
        ('clf', OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None), n_jobs=-1)),
    ])
    parameters = {
        'clf__estimator__alpha': (1e-2, 1e-3)
    }
    clf = grid_search(x_train, y_train, x_test, y_test, parameters, pipeline)
    return clf

In [9]:
def svm(x_train, x_test, y_train, y_test):


    print ("LinearSVC")
    pipeline = Pipeline([
        ('clf', OneVsRestClassifier(SVC(), n_jobs=-1)),
    ])
    
    parameters ={
         'clf__estimator__kernel': ['rbf'],
         'clf__estimator__gamma': [1e-3],
         'clf__estimator__C': [10]
        }
        
    clf = grid_search(x_train, y_train, x_test, y_test, parameters, pipeline)    
    return clf

## 1.Word2Vec

### 1.1 Reading The data

In [10]:
x_data = pd.read_csv('/home/sina/input/Word2Vec/x_embeddings_Word2Vec.csv')
y_data = pd.read_csv('/home/sina/input/Word2Vec/y_data_Word2Vec.csv')
x_data = x_data.as_matrix()
y_data = y_data.as_matrix()
x_data = x_data[0:200000]
y_data = y_data[0:200000]
print('x_data shape: ', x_data.shape)
print('y_data shape: ', y_data.shape)

x_data shape:  (200000, 300)
y_data shape:  (200000, 28)


#### 1.2 Split data to test and train

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.33, random_state=42)

#### 1.3 shape of train and test

In [12]:
print('x_train shape: ', x_train.shape)
print('y_train shape: ', y_train.shape)
print('x_test shape: ', x_test.shape)
print('y_test shape: ', y_test.shape)

x_train shape:  (134000, 300)
y_train shape:  (134000, 28)
x_test shape:  (66000, 300)
y_test shape:  (66000, 28)


### 1.4 apply different algorithm

#### 1.4.1 Logistic regression

##### 1.4.1.1 OneVsRestClassifier

In [111]:
bst_clf_logisticregression_OneVsRestClassifier_Word2Vec=logisticRegression(x_train, x_test, y_train, y_test)
save_model(bst_clf_logisticregression_OneVsRestClassifier_Word2Vec,'logisticregression_OneVsRestClassifier_Word2Vec.sav','Word2Vec')
prediction_logisticregression_OneVsRestClassifier_Word2Vec = bst_clf_logisticregression_OneVsRestClassifier_Word2Vec.predict(x_test)

LogisticRegression
Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] clf__estimator__C=0.1, clf__estimator__class_weight=None ........
[CV] clf__estimator__C=0.1, clf__estimator__class_weight=None ........
[CV] clf__estimator__C=1, clf__estimator__class_weight=None ..........
[CV]  clf__estimator__C=0.1, clf__estimator__class_weight=None, score=0.1017910447761194, total=   9.4s
[CV] clf__estimator__C=1, clf__estimator__class_weight=None ..........
[CV]  clf__estimator__C=0.1, clf__estimator__class_weight=None, score=0.09970149253731343, total=   9.6s
[CV] clf__estimator__C=10, clf__estimator__class_weight=None .........


[Parallel(n_jobs=3)]: Done   2 out of   6 | elapsed:    9.8s remaining:   19.6s


[CV]  clf__estimator__C=1, clf__estimator__class_weight=None, score=0.15417910447761193, total=  12.6s
[CV] clf__estimator__C=10, clf__estimator__class_weight=None .........


[Parallel(n_jobs=3)]: Done   3 out of   6 | elapsed:   12.9s remaining:   12.9s


[CV]  clf__estimator__C=1, clf__estimator__class_weight=None, score=0.1591044776119403, total=  12.5s


[Parallel(n_jobs=3)]: Done   4 out of   6 | elapsed:   22.1s remaining:   11.1s


[CV]  clf__estimator__C=10, clf__estimator__class_weight=None, score=0.15940298507462686, total=  37.6s
[CV]  clf__estimator__C=10, clf__estimator__class_weight=None, score=0.15895522388059702, total=  37.4s


[Parallel(n_jobs=3)]: Done   6 out of   6 | elapsed:   50.4s finished
[Parallel(n_jobs=3)]: Done   6 out of   6 | elapsed:   50.4s remaining:    0.0s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=-1))]
Applying best classifier on test data:
grid_search_tune.best_estimator :  ('clf', OneVsRestClassifier(estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=-1))


##### 1.4.1.2 ClassifierChain

In [112]:
chains = [ClassifierChain(LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', solver='sag', tol=0.0001,
          verbose=0, warm_start=False), order='random', random_state=i)
          for i in range(10)]

for chain in chains:
    chain.fit(x_train, y_train)

ClassifierChain(base_estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=0)

ClassifierChain(base_estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=1)

ClassifierChain(base_estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=2)

ClassifierChain(base_estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=3)

ClassifierChain(base_estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=4)

ClassifierChain(base_estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=5)

ClassifierChain(base_estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=6)

ClassifierChain(base_estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=7)

ClassifierChain(base_estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=8)

ClassifierChain(base_estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=9)

In [113]:
y_pred_chains = np.array([chain.predict(x_test) for chain in
                          chains])
predict_logisticregression_ensemble_ClassifierChain_Word2Vec = y_pred_chains.mean(axis=0)

temp = []
org = []
for i in range(len(predict_logisticregression_ensemble_ClassifierChain_Word2Vec)):
    for j in range(len(predict_logisticregression_ensemble_ClassifierChain_Word2Vec[0])):
        temp.append(int(predict_logisticregression_ensemble_ClassifierChain_Word2Vec[i][j]+0.8))
    org.append(temp)
    temp = []
    
predict_logisticregression_ensemble_ClassifierChain_Word2Vec = np.asarray(org) 

In [114]:
cc = CC(h=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', solver='sag', tol=0.0001,
          verbose=0, warm_start=False))
cc.fit(x_train,y_train)
save_model(cc,'LogisticRegression_Word2Vec_CC.sav','Word2Vec')

In [115]:
predict_logisticregression_CC_ClassifierChain_Word2Vec=cc.predict(x_test)

In [116]:
br = BR(h=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', solver='sag', tol=0.0001,
          verbose=0, warm_start=False))
br.fit(x_train,y_train)
save_model(br,'LogisticRegression_Word2Vec_BR.sav','Word2Vec')

In [117]:
predict_logisticregression_BR_ClassifierChain_Word2Vec=br.predict(x_test)

In [118]:
mcc = MCC(h=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', solver='sag', tol=0.0001,
          verbose=0, warm_start=False))
mcc.fit(x_train,y_train)
save_model(mcc,'LogisticRegression_Word2Vec_MCC.sav','Word2Vec')

In [119]:
predict_logisticregression_MCC_ClassifierChain_Word2Vec=mcc.predict(x_test)

##### 1.4.1.3 visualization and scores

In [121]:
f1_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec = f1_score(y_test,prediction_logisticregression_OneVsRestClassifier_Word2Vec,average='weighted')
f1_score_prediction_logisticregression_pure_ClassifierChain_Word2Vec = f1_score(y_test,predict_logisticregression_ensemble_ClassifierChain_Word2Vec,average='weighted')
f1_score_logisticregression_CC_ClassifierChain_Word2Vec = f1_score(y_test,predict_logisticregression_CC_ClassifierChain_Word2Vec,average='weighted')
f1_score_logisticregression_BR_ClassifierChain_Word2Vec = f1_score(y_test,predict_logisticregression_BR_ClassifierChain_Word2Vec,average='weighted')
f1_score_logisticregression_MCC_ClassifierChain_Word2Vec = f1_score(y_test,predict_logisticregression_MCC_ClassifierChain_Word2Vec,average='weighted')

print('f1_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec: ',f1_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec)
print('f1_score_prediction_logisticregression_pure_chain_Word2Vec: ',f1_score_prediction_logisticregression_pure_ClassifierChain_Word2Vec)
print('f1_score_logisticregression_CC_ClassifierChain_Word2Vec: ',f1_score_logisticregression_CC_ClassifierChain_Word2Vec)
print('f1_score_logisticregression_BR_ClassifierChain_Word2Vec: ',f1_score_logisticregression_BR_ClassifierChain_Word2Vec)
print('f1_score_logisticregression_MCC_ClassifierChain_Word2Vec: ',f1_score_logisticregression_MCC_ClassifierChain_Word2Vec)

f1_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec:  0.458769941443
f1_score_prediction_logisticregression_pure_chain_Word2Vec:  0.495937437133
f1_score_logisticregression_CC_ClassifierChain_Word2Vec:  0.465931111214
f1_score_logisticregression_BR_ClassifierChain_Word2Vec:  0.458790223185
f1_score_logisticregression_MCC_ClassifierChain_Word2Vec:  0.468526692773


In [122]:
Hamming_loss_prediction_logisticregression_OneVsRestClassifier_Word2Vec = Hamming_loss(y_test,prediction_logisticregression_OneVsRestClassifier_Word2Vec)
Hamming_loss_prediction_logisticregression_pure_ClassifierChain_Word2Vec = Hamming_loss(y_test,predict_logisticregression_ensemble_ClassifierChain_Word2Vec)
Hamming_loss_logisticregression_CC_ClassifierChain_Word2Vec = Hamming_loss(y_test,predict_logisticregression_CC_ClassifierChain_Word2Vec)
Hamming_loss_logisticregression_BR_ClassifierChain_Word2Vec = Hamming_loss(y_test,predict_logisticregression_BR_ClassifierChain_Word2Vec)
Hamming_loss_logisticregression_MCC_ClassifierChain_Word2Vec = Hamming_loss(y_test,predict_logisticregression_MCC_ClassifierChain_Word2Vec)

print('Hamming_loss_prediction_logisticregression_OneVsRestClassifier_Word2Vec: ',Hamming_loss_prediction_logisticregression_OneVsRestClassifier_Word2Vec)
print('Hamming_loss_prediction_logisticregression_pure_chain_Word2Vec: ',Hamming_loss_prediction_logisticregression_pure_ClassifierChain_Word2Vec)
print('Hamming_loss_logisticregression_CC_ClassifierChain_Word2Vec: ',Hamming_loss_logisticregression_CC_ClassifierChain_Word2Vec)
print('Hamming_loss_logisticregression_BR_ClassifierChain_Word2Vec: ',Hamming_loss_logisticregression_BR_ClassifierChain_Word2Vec)
print('Hamming_loss_logisticregression_MCC_ClassifierChain_Word2Vec: ',Hamming_loss_logisticregression_MCC_ClassifierChain_Word2Vec)

Hamming_loss_prediction_logisticregression_OneVsRestClassifier_Word2Vec:  0.0633441558442
Hamming_loss_prediction_logisticregression_pure_chain_Word2Vec:  0.0659686147186
Hamming_loss_logisticregression_CC_ClassifierChain_Word2Vec:  0.0643019480519
Hamming_loss_logisticregression_BR_ClassifierChain_Word2Vec:  0.063382034632
Hamming_loss_logisticregression_MCC_ClassifierChain_Word2Vec:  0.0641829004329


In [123]:
Hamming_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec = Hamming_score(y_test,prediction_logisticregression_OneVsRestClassifier_Word2Vec)
Hamming_score_prediction_logisticregression_pure_ClassifierChain_Word2Vec = Hamming_score(y_test,predict_logisticregression_ensemble_ClassifierChain_Word2Vec)
Hamming_score_logisticregression_CC_ClassifierChain_Word2Vec = Hamming_score(y_test,predict_logisticregression_CC_ClassifierChain_Word2Vec)
Hamming_score_logisticregression_BR_ClassifierChain_Word2Vec = Hamming_score(y_test,predict_logisticregression_BR_ClassifierChain_Word2Vec)
Hamming_score_logisticregression_MCC_ClassifierChain_Word2Vec = Hamming_score(y_test,predict_logisticregression_MCC_ClassifierChain_Word2Vec)

print('Hamming_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec: ',Hamming_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec)
print('Hamming_score_prediction_logisticregression_pure_chain_Word2Vec: ',Hamming_score_prediction_logisticregression_pure_ClassifierChain_Word2Vec)
print('Hamming_score_logisticregression_CC_ClassifierChain_Word2Vec: ',Hamming_score_logisticregression_CC_ClassifierChain_Word2Vec)
print('Hamming_score_logisticregression_BR_ClassifierChain_Word2Vec: ',Hamming_score_logisticregression_BR_ClassifierChain_Word2Vec)
print('Hamming_score_logisticregression_MCC_ClassifierChain_Word2Vec: ',Hamming_score_logisticregression_MCC_ClassifierChain_Word2Vec)

Hamming_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec:  0.936655844156
Hamming_score_prediction_logisticregression_pure_chain_Word2Vec:  0.934031385281
Hamming_score_logisticregression_CC_ClassifierChain_Word2Vec:  0.935698051948
Hamming_score_logisticregression_BR_ClassifierChain_Word2Vec:  0.936617965368
Hamming_score_logisticregression_MCC_ClassifierChain_Word2Vec:  0.935817099567


In [124]:
Exact_match_prediction_logisticregression_OneVsRestClassifier_Word2Vec = Exact_match(y_test,prediction_logisticregression_OneVsRestClassifier_Word2Vec)
Exact_match_prediction_logisticregression_pure_ClassifierChain_Word2Vec = Exact_match(y_test,predict_logisticregression_ensemble_ClassifierChain_Word2Vec)
Exact_match_logisticregression_CC_ClassifierChain_Word2Vec = Exact_match(y_test,predict_logisticregression_CC_ClassifierChain_Word2Vec)
Exact_match_logisticregression_BR_ClassifierChain_Word2Vec = Exact_match(y_test,predict_logisticregression_BR_ClassifierChain_Word2Vec)
Exact_match_logisticregression_MCC_ClassifierChain_Word2Vec = Exact_match(y_test,predict_logisticregression_MCC_ClassifierChain_Word2Vec)

print('Exact_match_prediction_logisticregression_OneVsRestClassifier_Word2Vec: ',Exact_match_prediction_logisticregression_OneVsRestClassifier_Word2Vec)
print('Exact_match_prediction_logisticregression_pure_chain_Word2Vec: ',Exact_match_prediction_logisticregression_pure_ClassifierChain_Word2Vec)
print('Exact_match_logisticregression_CC_ClassifierChain_Word2Vec: ',Exact_match_logisticregression_CC_ClassifierChain_Word2Vec)
print('Exact_match_logisticregression_BR_ClassifierChain_Word2Vec: ',Exact_match_logisticregression_BR_ClassifierChain_Word2Vec)
print('Exact_match_logisticregression_MCC_ClassifierChain_Word2Vec: ',Exact_match_logisticregression_MCC_ClassifierChain_Word2Vec)

Exact_match_prediction_logisticregression_OneVsRestClassifier_Word2Vec:  0.160454545455
Exact_match_prediction_logisticregression_pure_chain_Word2Vec:  0.17696969697
Exact_match_logisticregression_CC_ClassifierChain_Word2Vec:  0.186666666667
Exact_match_logisticregression_BR_ClassifierChain_Word2Vec:  0.161060606061
Exact_match_logisticregression_MCC_ClassifierChain_Word2Vec:  0.19303030303


#### 1.4.2 DecisionTreeClassifier

In [125]:
DecisionTreeClassifier_Word2Vec = DecisionTreeClassifier(min_samples_split=2, random_state=1)
DecisionTreeClassifier_Word2Vec.fit(x_train,y_train)
save_model(DecisionTreeClassifier_Word2Vec,'DecisionTreeClassifier_Word2Vec.sav','Word2Vec')

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [126]:
predict_DecisionTreeClassifier_Word2Vec = DecisionTreeClassifier_Word2Vec.predict(x_test)

In [127]:
f1_score_DecisionTreeClassifier_Word2Vec = f1_score(y_test,predict_DecisionTreeClassifier_Word2Vec,average='weighted')

print('f1_score_prediction_DecisionTreeClassifier_Word2Vec: ',f1_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec)

f1_score_prediction_DecisionTreeClassifier_Word2Vec:  0.458769941443


In [128]:
Hamming_loss_DecisionTreeClassifier_Word2Vec = Hamming_loss(y_test,predict_DecisionTreeClassifier_Word2Vec)

print('Hamming_loss_DecisionTreeClassifier_Word2Vec: ',Hamming_loss_DecisionTreeClassifier_Word2Vec)


Hamming_loss_DecisionTreeClassifier_Word2Vec:  0.10987012987


In [129]:
Hamming_score_DecisionTreeClassifier_Word2Vec = Hamming_score(y_test,predict_DecisionTreeClassifier_Word2Vec)

print('Hamming_score_DecisionTreeClassifier_Word2Vec: ',Hamming_score_DecisionTreeClassifier_Word2Vec)


Hamming_score_DecisionTreeClassifier_Word2Vec:  0.89012987013


In [130]:
Exact_match_DecisionTreeClassifier_Word2Vec = Exact_match(y_test,predict_DecisionTreeClassifier_Word2Vec)

print('Exact_match_DecisionTreeClassifier_Word2Vec: ',Exact_match_DecisionTreeClassifier_Word2Vec)


Exact_match_DecisionTreeClassifier_Word2Vec:  0.0557575757576


#### 1.4.4 SVM

##### 1.4.4.1 OneVsRestClassifier

In [131]:
clf_SVM_OneVsRestClassifier_Word2Vec = svm(x_train, x_test, y_train, y_test)
prediction_SVM_OneVsRestClassifier_Word2Vec = clf_SVM_OneVsRestClassifier_Word2Vec.predict(x_test)
save_model(clf_SVM_OneVsRestClassifier_Word2Vec,'SVM_OneVsRestClassifier_Word2Vec.sav','Word2Vec')

LinearSVC
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] clf__estimator__C=10, clf__estimator__gamma=0.001, clf__estimator__kernel=rbf 
[CV] clf__estimator__C=10, clf__estimator__gamma=0.001, clf__estimator__kernel=rbf 
[CV]  clf__estimator__C=10, clf__estimator__gamma=0.001, clf__estimator__kernel=rbf, score=0.0032835820895522386, total= 4.3min
[CV]  clf__estimator__C=10, clf__estimator__gamma=0.001, clf__estimator__kernel=rbf, score=0.003582089552238806, total= 4.4min


[Parallel(n_jobs=3)]: Done   2 out of   2 | elapsed:  6.3min remaining:    0.0s
[Parallel(n_jobs=3)]: Done   2 out of   2 | elapsed:  6.3min finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=-1))]
Applying best classifier on test data:
grid_search_tune.best_estimator :  ('clf', OneVsRestClassifier(estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=-1))


##### 1.4.4.2 ClassifierChain

In [132]:
chains = [ClassifierChain(SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, shrinking=True,
  tol=0.001, verbose=False), random_state=i)
          for i in range(10)]

for chain in chains:
    chain.fit(x_train, y_train)

ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=0)

ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=1)

ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=2)

ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=3)

ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=4)

ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=5)

ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=6)

ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=7)

ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=8)

ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=9)

In [133]:
y_pred_chains = np.array([chain.predict(x_test) for chain in
                          chains])
predict_svm_ensemble_ClassifierChain_Word2Vec = y_pred_chains.mean(axis=0)



temp = []
org = []
for i in range(len(predict_svm_ensemble_ClassifierChain_Word2Vec)):
    for j in range(len(predict_svm_ensemble_ClassifierChain_Word2Vec[0])):
        temp.append(int(predict_svm_ensemble_ClassifierChain_Word2Vec[i][j]+0.8))
    org.append(temp)
    temp = []
    
predict_svm_ensemble_ClassifierChain_Word2Vec = np.asarray(org)

In [134]:
cc = CC(h=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, shrinking=True,
  tol=0.001, verbose=False))
cc.fit(x_train,y_train)
save_model(cc,'SVM_Chain_CC_Word2Vec.sav','Word2Vec')

In [135]:
br = BR(h=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, shrinking=True,
  tol=0.001, verbose=False))
br.fit(x_train,y_train)
save_model(br,'SVM_Chain_BR_Word2Vec.sav','Word2Vec')

In [ ]:
mcc = MCC(h=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, shrinking=True,
  tol=0.001, verbose=False))
mcc.fit(x_train,y_train)
save_model(mcc,'SVM_Chain_MCC_Word2Vec.sav','Word2Vec')

In [ ]:
predict_svm_CC_ClassifierChain_Word2Vec=cc.predict(x_test)
predict_svm_BR_ClassifierChain_Word2Vec=br.predict(x_test)
predict_svm_MCC_ClassifierChain_Word2Vec=mcc.predict(x_test)

In [ ]:
f1_score_svm_OnevsResClassifier_svm_Word2Vec = f1_score(y_test,prediction_SVM_OneVsRestClassifier_Word2Vec,average='weighted')
f1_score_svm_pure_chain_svm_Word2Vec = f1_score(y_test,predict_svm_ensemble_ClassifierChain_Word2Vec,average='weighted')
f1_score_svm_CC_svm_Word2Vec = f1_score(y_test,predict_svm_CC_ClassifierChain_Word2Vec,average='weighted')
f1_score_svm_BR_svm_Word2Vec = f1_score(y_test,predict_svm_BR_ClassifierChain_Word2Vec,average='weighted')
f1_score_svm_MCC_svm_Word2Vec = f1_score(y_test,predict_svm_MCC_ClassifierChain_Word2Vec,average='weighted')

print('f1_score_svm_OnevsResClassifier_svm_Word2Vec: ',f1_score_svm_OnevsResClassifier_svm_Word2Vec)
print('f1_score_svm_pure_chain_svm_Word2Vec: ',f1_score_svm_pure_chain_svm_Word2Vec)
print('f1_score_svm_CC_ClassifierChain_Word2Vec: ',f1_score_svm_CC_svm_Word2Vec)
print('f1_score_svm_BR_ClassifierChain_Word2Vec: ',f1_score_svm_BR_svm_Word2Vec)
print('f1_score_svm_MCC_ClassifierChain_Word2Vec: ',f1_score_svm_MCC_svm_Word2Vec)

In [ ]:
Hamming_loss_svm_OnevsResClassifier_svm_Word2Vec = Hamming_loss(y_test,prediction_SVM_OneVsRestClassifier_Word2Vec)
Hamming_loss_svm_pure_chain_svm_Word2Vec = Hamming_loss(y_test,predict_svm_ensemble_ClassifierChain_Word2Vec)
Hamming_loss_svm_CC_svm_Word2Vec = Hamming_loss(y_test,predict_svm_CC_ClassifierChain_Word2Vec)
Hamming_loss_svm_BR_svm_Word2Vec = Hamming_loss(y_test,predict_svm_BR_ClassifierChain_Word2Vec)
Hamming_loss_svm_MCC_svm_Word2Vec = Hamming_loss(y_test,predict_svm_MCC_ClassifierChain_Word2Vec)

print('Hamming_loss_svm_OnevsResClassifier_svm_Word2Vec: ',Hamming_loss_svm_OnevsResClassifier_svm_Word2Vec)
print('Hamming_loss_svm_pure_chain_svm_Word2Vec: ',Hamming_loss_svm_pure_chain_svm_Word2Vec)
print('Hamming_loss_svm_CC_ClassifierChain_Word2Vec: ',Hamming_loss_svm_CC_svm_Word2Vec)
print('Hamming_loss_svm_BR_ClassifierChain_Word2Vec: ',Hamming_loss_svm_BR_svm_Word2Vec)
print('Hamming_loss_svm_MCC_ClassifierChain_Word2Vec: ',Hamming_loss_svm_MCC_svm_Word2Vec)

In [ ]:
Hamming_score_svm_OnevsResClassifier_svm_Word2Vec = Hamming_score(y_test,prediction_SVM_OneVsRestClassifier_Word2Vec)
Hamming_score_svm_pure_chain_svm_Word2Vec = Hamming_score(y_test,predict_svm_ensemble_ClassifierChain_Word2Vec)
Hamming_score_svm_CC_svm_Word2Vec = Hamming_score(y_test,predict_svm_CC_ClassifierChain_Word2Vec)
Hamming_score_svm_BR_svm_Word2Vec = Hamming_score(y_test,predict_svm_BR_ClassifierChain_Word2Vec)
Hamming_score_svm_MCC_svm_Word2Vec = Hamming_score(y_test,predict_svm_MCC_ClassifierChain_Word2Vec)

print('Hamming_score_svm_OnevsResClassifier_svm_Word2Vec: ',Hamming_score_svm_OnevsResClassifier_svm_Word2Vec)
print('Hamming_score_svm_pure_chain_svm_Word2Vec: ',Hamming_score_svm_pure_chain_svm_Word2Vec)
print('Hamming_score_svm_CC_ClassifierChain_Word2Vec: ',Hamming_score_svm_CC_svm_Word2Vec)
print('Hamming_score_svm_BR_ClassifierChain_Word2Vec: ',Hamming_score_svm_BR_svm_Word2Vec)
print('Hamming_score_svm_MCC_ClassifierChain_Word2Vec: ',Hamming_score_svm_MCC_svm_Word2Vec)

In [ ]:
Exact_match_svm_OnevsResClassifier_svm_Word2Vec = Exact_match(y_test,prediction_SVM_OneVsRestClassifier_Word2Vec)
Exact_match_svm_pure_chain_svm_Word2Vec = Exact_match(y_test,predict_svm_ensemble_ClassifierChain_Word2Vec)
Exact_match_svm_CC_svm_Word2Vec = Exact_match(y_test,predict_svm_CC_ClassifierChain_Word2Vec)
Exact_match_svm_BR_svm_Word2Vec = Exact_match(y_test,predict_svm_BR_ClassifierChain_Word2Vec)
Exact_match_svm_MCC_svm_Word2Vec = Exact_match(y_test,predict_svm_MCC_ClassifierChain_Word2Vec)

print('Exact_match_svm_OnevsResClassifier_svm_Word2Vec: ',Exact_match_svm_OnevsResClassifier_svm_Word2Vec)
print('Exact_match_svm_pure_chain_svm_Word2Vec: ',Exact_match_svm_pure_chain_svm_Word2Vec)
print('Exact_match_svm_CC_ClassifierChain_Word2Vec: ',Exact_match_svm_CC_svm_Word2Vec)
print('Exact_match_svm_BR_ClassifierChain_Word2Vec: ',Exact_match_svm_BR_svm_Word2Vec)
print('Exact_match_svm_MCC_ClassifierChain_Word2Vec: ',Exact_match_svm_MCC_svm_Word2Vec)

## 2.Word2Vec_top10

### 2.1 Reading The data


In [18]:
x_data_10 = pd.read_csv('/home/sina/input/word2vec_10/x_embeddings_Word2Vec_top10.csv')
y_data_10 = pd.read_csv('/home/sina/input/word2vec_10/y_data_Word2Vec_top10.csv')
x_data_10 = x_data_10.as_matrix()
y_data_10 = y_data_10.as_matrix()
x_data_10 = x_data_10[0:200000]
y_data_10 = y_data_10[0:200000]
print('x_data_10 shape: ', x_data_10.shape)
print('y_data_10 shape: ', y_data_10.shape)

x_data_10 shape:  (200000, 300)
y_data_10 shape:  (200000, 28)


In [19]:
y_data_10 = np.delete(y_data_10,22,1)

#### 2.2 Split data to test and train

In [20]:
type(y_data_10)

numpy.ndarray

In [21]:
x_train_10, x_test_10, y_train_10, y_test_10 = train_test_split(x_data_10, y_data_10, test_size=0.33, random_state=4)

#### 2.3 shape of train and test

In [22]:
print('x_train_10 shape: ', x_train_10.shape)
print('y_train_10 shape: ', y_train_10.shape)
print('x_test_10 shape: ', x_test_10.shape)
print('y_test_10 shape: ', y_test_10.shape)

x_train_10 shape:  (134000, 300)
y_train_10 shape:  (134000, 27)
x_test_10 shape:  (66000, 300)
y_test_10 shape:  (66000, 27)


### 2.4 apply different algorithm


#### 2.4.1 Logistic regression

##### 2.4.1.1 OneVsRestClassifier


In [30]:
bst_clf_logisticregression_OneVsRestClassifier_Word2Vec_10=logisticRegression(x_train_10, x_test_10, y_train_10, y_test_10)
save_model(bst_clf_logisticregression_OneVsRestClassifier_Word2Vec_10,'logisticregression_OneVsRestClassifier_Word2Vec_10.sav','Word2Vec_10')
prediction_logisticregression_OneVsRestClassifier_Word2Vec_10 = bst_clf_logisticregression_OneVsRestClassifier_Word2Vec_10.predict(x_test_10)

LogisticRegression
Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] clf__estimator__C=0.1, clf__estimator__class_weight=None ........
[CV] clf__estimator__C=0.1, clf__estimator__class_weight=None ........
[CV] clf__estimator__C=1, clf__estimator__class_weight=None ..........
[CV]  clf__estimator__C=0.1, clf__estimator__class_weight=None, score=0.05925373134328358, total=   9.6s
[CV] clf__estimator__C=1, clf__estimator__class_weight=None ..........
[CV]  clf__estimator__C=0.1, clf__estimator__class_weight=None, score=0.0635820895522388, total=   9.8s
[CV] clf__estimator__C=10, clf__estimator__class_weight=None .........


[Parallel(n_jobs=3)]: Done   2 out of   6 | elapsed:   10.0s remaining:   20.1s


[CV]  clf__estimator__C=1, clf__estimator__class_weight=None, score=0.19850746268656716, total=  12.4s
[CV] clf__estimator__C=10, clf__estimator__class_weight=None .........


[Parallel(n_jobs=3)]: Done   3 out of   6 | elapsed:   12.8s remaining:   12.8s


[CV]  clf__estimator__C=1, clf__estimator__class_weight=None, score=0.19298507462686568, total=  12.3s


[Parallel(n_jobs=3)]: Done   4 out of   6 | elapsed:   22.3s remaining:   11.2s


[CV]  clf__estimator__C=10, clf__estimator__class_weight=None, score=0.24223880597014924, total=  35.7s
[CV]  clf__estimator__C=10, clf__estimator__class_weight=None, score=0.24865671641791046, total=  44.7s


[Parallel(n_jobs=3)]: Done   6 out of   6 | elapsed:   54.8s remaining:    0.0s
[Parallel(n_jobs=3)]: Done   6 out of   6 | elapsed:   54.8s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=-1))]
Applying best classifier on test data:
grid_search_tune.best_estimator :  ('clf', OneVsRestClassifier(estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=-1))


##### 2.4.1.2 ClassifierChain

In [66]:
chains = [ClassifierChain(LogisticRegression(C=30, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', solver='sag', tol=0.0001,
          verbose=0, warm_start=False), order='random', random_state=i)
          for i in range(10)]
i=0
for chain in chains:
    i+=1
    print(i)
    chain.fit(x_train_10, y_train_10)

1


ClassifierChain(base_estimator=LogisticRegression(C=30, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=0)

2


ClassifierChain(base_estimator=LogisticRegression(C=30, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=1)

3


...

ClassifierChain(base_estimator=LogisticRegression(C=30, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=7)

9


ClassifierChain(base_estimator=LogisticRegression(C=30, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=8)

10


ClassifierChain(base_estimator=LogisticRegression(C=30, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False),
        cv=None, order='random', random_state=9)

In [67]:
y_pred_chains = np.array([chain.predict(x_test_10) for chain in
                          chains])
predict_logisticregression_ensemble_ClassifierChain_Word2Vec_10 = y_pred_chains.mean(axis=0)

temp = []
org = []
for i in range(len(predict_logisticregression_ensemble_ClassifierChain_Word2Vec_10)):
    for j in range(len(predict_logisticregression_ensemble_ClassifierChain_Word2Vec_10[0])):
        temp.append(int(predict_logisticregression_ensemble_ClassifierChain_Word2Vec_10[i][j]+0.8))
    org.append(temp)
    temp = []
    
predict_logisticregression_ensemble_ClassifierChain_Word2Vec_10 = np.asarray(org) 

In [68]:
cc = CC(h=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', solver='sag', tol=0.0001,
          verbose=0, warm_start=False))
cc.fit(x_train_10,y_train_10)
save_model(cc,'LogisticRegression_Word2Vec_CC_10.sav','Word2Vec_10')

In [69]:
predict_logisticregression_CC_ClassifierChain_Word2Vec_10=cc.predict(x_test_10)

In [70]:
br = BR(h=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', solver='sag', tol=0.0001,
          verbose=0, warm_start=False))
br.fit(x_train_10,y_train_10)
save_model(br,'LogisticRegression_Word2Vec_BR_10.sav','Word2Vec_10')

In [71]:
predict_logisticregression_BR_ClassifierChain_Word2Vec_10=br.predict(x_test_10)

In [72]:
mcc = MCC(h=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', solver='sag', tol=0.0001,
          verbose=0, warm_start=False))
mcc.fit(x_train_10,y_train_10)
save_model(mcc,'LogisticRegression_Word2Vec_MCC.sav_10','Word2Vec_10')

In [73]:
predict_logisticregression_MCC_ClassifierChain_Word2Vec_10=mcc.predict(x_test_10)

##### 2.4.1.3 visualization and scores


In [74]:
predict_logisticregression_ensemble_ClassifierChain_Word2Vec_10.shape

(6600, 27)

In [75]:
predict_logisticregression_ensemble_ClassifierChain_Word2Vec_10.shape

(6600, 27)

In [76]:
f1_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10 = f1_score(y_test_10,prediction_logisticregression_OneVsRestClassifier_Word2Vec_10,average='weighted')
f1_score_prediction_logisticregression_pure_ClassifierChain_Word2Vec_10 = f1_score(y_test_10,predict_logisticregression_ensemble_ClassifierChain_Word2Vec_10,average='weighted')
f1_score_logisticregression_CC_ClassifierChain_Word2Vec_10 = f1_score(y_test_10,predict_logisticregression_CC_ClassifierChain_Word2Vec_10,average='weighted')
f1_score_logisticregression_BR_ClassifierChain_Word2Vec_10 = f1_score(y_test_10,predict_logisticregression_BR_ClassifierChain_Word2Vec_10,average='weighted')
f1_score_logisticregression_MCC_ClassifierChain_Word2Vec_10 = f1_score(y_test_10,predict_logisticregression_MCC_ClassifierChain_Word2Vec_10,average='weighted')

print('f1_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10: ',f1_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10)
print('f1_score_prediction_logisticregression_pure_chain_Word2Vec_10: ',f1_score_prediction_logisticregression_pure_ClassifierChain_Word2Vec_10)
print('f1_score_logisticregression_CC_ClassifierChain_Word2Vec_10: ',f1_score_logisticregression_CC_ClassifierChain_Word2Vec_10)
print('f1_score_logisticregression_BR_ClassifierChain_Word2Vec_10: ',f1_score_logisticregression_BR_ClassifierChain_Word2Vec_10)
print('f1_score_logisticregression_MCC_ClassifierChain_Word2Vec_10: ',f1_score_logisticregression_MCC_ClassifierChain_Word2Vec_10)

f1_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10:  0.436735826424
f1_score_prediction_logisticregression_pure_chain_Word2Vec_10:  0.499217589218
f1_score_logisticregression_CC_ClassifierChain_Word2Vec_10:  0.464418432692
f1_score_logisticregression_BR_ClassifierChain_Word2Vec_10:  0.43704132753
f1_score_logisticregression_MCC_ClassifierChain_Word2Vec_10:  0.492505577493


In [77]:
Hamming_loss_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10 = Hamming_loss(y_test_10,prediction_logisticregression_OneVsRestClassifier_Word2Vec_10)
Hamming_loss_prediction_logisticregression_pure_ClassifierChain_Word2Vec_10 = Hamming_loss(y_test_10,predict_logisticregression_ensemble_ClassifierChain_Word2Vec_10)
Hamming_loss_logisticregression_CC_ClassifierChain_Word2Vec_10 = Hamming_loss(y_test_10,predict_logisticregression_CC_ClassifierChain_Word2Vec_10)
Hamming_loss_logisticregression_BR_ClassifierChain_Word2Vec_10 = Hamming_loss(y_test_10,predict_logisticregression_BR_ClassifierChain_Word2Vec_10)
Hamming_loss_logisticregression_MCC_ClassifierChain_Word2Vec_10 = Hamming_loss(y_test_10,predict_logisticregression_MCC_ClassifierChain_Word2Vec_10)

print('Hamming_loss_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10: ',Hamming_loss_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10)
print('Hamming_loss_prediction_logisticregression_pure_chain_Word2Vec_10: ',Hamming_loss_prediction_logisticregression_pure_ClassifierChain_Word2Vec_10)
print('Hamming_loss_logisticregression_CC_ClassifierChain_Word2Vec_10: ',Hamming_loss_logisticregression_CC_ClassifierChain_Word2Vec_10)
print('Hamming_loss_logisticregression_BR_ClassifierChain_Word2Vec_10: ',Hamming_loss_logisticregression_BR_ClassifierChain_Word2Vec_10)
print('Hamming_loss_logisticregression_MCC_ClassifierChain_Word2Vec_10: ',Hamming_loss_logisticregression_MCC_ClassifierChain_Word2Vec_10)

Hamming_loss_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10:  0.0396801346801
Hamming_loss_prediction_logisticregression_pure_chain_Word2Vec_10:  0.0516722783389
Hamming_loss_logisticregression_CC_ClassifierChain_Word2Vec_10:  0.0445230078563
Hamming_loss_logisticregression_BR_ClassifierChain_Word2Vec_10:  0.0396240179574
Hamming_loss_logisticregression_MCC_ClassifierChain_Word2Vec_10:  0.0420145903479


In [78]:
Hamming_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10 = Hamming_score(y_test_10,prediction_logisticregression_OneVsRestClassifier_Word2Vec_10)
Hamming_score_prediction_logisticregression_pure_ClassifierChain_Word2Vec_10 = Hamming_score(y_test_10,predict_logisticregression_ensemble_ClassifierChain_Word2Vec_10)
Hamming_score_logisticregression_CC_ClassifierChain_Word2Vec_10 = Hamming_score(y_test_10,predict_logisticregression_CC_ClassifierChain_Word2Vec_10)
Hamming_score_logisticregression_BR_ClassifierChain_Word2Vec_10 = Hamming_score(y_test_10,predict_logisticregression_BR_ClassifierChain_Word2Vec_10)
Hamming_score_logisticregression_MCC_ClassifierChain_Word2Vec_10 = Hamming_score(y_test_10,predict_logisticregression_MCC_ClassifierChain_Word2Vec_10)

print('Hamming_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10: ',Hamming_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10)
print('Hamming_score_prediction_logisticregression_pure_chain_Word2Vec_10: ',Hamming_score_prediction_logisticregression_pure_ClassifierChain_Word2Vec_10)
print('Hamming_score_logisticregression_CC_ClassifierChain_Word2Vec_10: ',Hamming_score_logisticregression_CC_ClassifierChain_Word2Vec_10)
print('Hamming_score_logisticregression_BR_ClassifierChain_Word2Vec_10: ',Hamming_score_logisticregression_BR_ClassifierChain_Word2Vec_10)
print('Hamming_score_logisticregression_MCC_ClassifierChain_Word2Vec_10: ',Hamming_score_logisticregression_MCC_ClassifierChain_Word2Vec_10)

Hamming_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10:  0.96031986532
Hamming_score_prediction_logisticregression_pure_chain_Word2Vec_10:  0.948327721661
Hamming_score_logisticregression_CC_ClassifierChain_Word2Vec_10:  0.955476992144
Hamming_score_logisticregression_BR_ClassifierChain_Word2Vec_10:  0.960375982043
Hamming_score_logisticregression_MCC_ClassifierChain_Word2Vec_10:  0.957985409652


In [79]:
Exact_match_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10 = Exact_match(y_test_10,prediction_logisticregression_OneVsRestClassifier_Word2Vec_10)
Exact_match_prediction_logisticregression_pure_ClassifierChain_Word2Vec_10 = Exact_match(y_test_10,predict_logisticregression_ensemble_ClassifierChain_Word2Vec_10)
Exact_match_logisticregression_CC_ClassifierChain_Word2Vec_10 = Exact_match(y_test_10,predict_logisticregression_CC_ClassifierChain_Word2Vec_10)
Exact_match_logisticregression_BR_ClassifierChain_Word2Vec_10 = Exact_match(y_test_10,predict_logisticregression_BR_ClassifierChain_Word2Vec_10)
Exact_match_logisticregression_MCC_ClassifierChain_Word2Vec_10 = Exact_match(y_test_10,predict_logisticregression_MCC_ClassifierChain_Word2Vec_10)

print('Exact_match_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10: ',Exact_match_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10)
print('Exact_match_prediction_logisticregression_pure_chain_Word2Vec_10: ',Exact_match_prediction_logisticregression_pure_ClassifierChain_Word2Vec_10)
print('Exact_match_logisticregression_CC_ClassifierChain_Word2Vec_10: ',Exact_match_logisticregression_CC_ClassifierChain_Word2Vec_10)
print('Exact_match_logisticregression_BR_ClassifierChain_Word2Vec_10: ',Exact_match_logisticregression_BR_ClassifierChain_Word2Vec_10)
print('Exact_match_logisticregression_MCC_ClassifierChain_Word2Vec_10: ',Exact_match_logisticregression_MCC_ClassifierChain_Word2Vec_10)

Exact_match_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10:  0.263181818182
Exact_match_prediction_logisticregression_pure_chain_Word2Vec_10:  0.287575757576
Exact_match_logisticregression_CC_ClassifierChain_Word2Vec_10:  0.34696969697
Exact_match_logisticregression_BR_ClassifierChain_Word2Vec_10:  0.263939393939
Exact_match_logisticregression_MCC_ClassifierChain_Word2Vec_10:  0.385151515152


#### 2.4.2 DecisionTreeClassifier

In [80]:
DecisionTreeClassifier_Word2Vec_10 = DecisionTreeClassifier(min_samples_split=2, random_state=1)
DecisionTreeClassifier_Word2Vec_10.fit(x_train_10,y_train_10)
save_model(DecisionTreeClassifier_Word2Vec_10,'DecisionTreeClassifier_Word2Vec_10.sav','Word2Vec_10')

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [82]:
predict_DecisionTreeClassifier_Word2Vec_10 = DecisionTreeClassifier_Word2Vec_10.predict(x_test_10)

In [83]:
f1_score_DecisionTreeClassifier_Word2Vec_10 = f1_score(y_test_10,predict_DecisionTreeClassifier_Word2Vec_10,average='weighted')

print('f1_score_prediction_DecisionTreeClassifier_Word2Vec_10: ',f1_score_prediction_logisticregression_OneVsRestClassifier_Word2Vec_10)

f1_score_prediction_DecisionTreeClassifier_Word2Vec_10:  0.436735826424


In [84]:
Hamming_loss_DecisionTreeClassifier_Word2Vec_10 = Hamming_loss(y_test_10,predict_DecisionTreeClassifier_Word2Vec_10)

print('Hamming_loss_DecisionTreeClassifier_Word2Vec_10: ',Hamming_loss_DecisionTreeClassifier_Word2Vec_10)


Hamming_loss_DecisionTreeClassifier_Word2Vec_10:  0.0708136924804


In [85]:
Hamming_score_DecisionTreeClassifier_Word2Vec_10 = Hamming_score(y_test_10,predict_DecisionTreeClassifier_Word2Vec_10)

print('Hamming_score_DecisionTreeClassifier_Word2Vec_10: ',Hamming_score_DecisionTreeClassifier_Word2Vec_10)


Hamming_score_DecisionTreeClassifier_Word2Vec_10:  0.92918630752


In [86]:
Exact_match_DecisionTreeClassifier_Word2Vec_10 = Exact_match(y_test_10,predict_DecisionTreeClassifier_Word2Vec_10)

print('Exact_match_DecisionTreeClassifier_Word2Vec_10: ',Exact_match_DecisionTreeClassifier_Word2Vec_10)


Exact_match_DecisionTreeClassifier_Word2Vec_10:  0.154242424242


#### 2.4.4 SVM

##### 2.4.4.1 OneVsRestClassifier


In [31]:
clf_SVM_OneVsRestClassifier_Word2Vec_10 = svm(x_train_10, x_test_10, y_train_10, y_test_10)
prediction_SVM_OneVsRestClassifier_Word2Vec_10 = clf_SVM_OneVsRestClassifier_Word2Vec_10.predict(x_test_10)
save_model(clf_SVM_OneVsRestClassifier_Word2Vec_10,'SVM_OneVsRestClassifier_Word2Vec_10.sav','Word2Vec_10')

LinearSVC
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] clf__estimator__C=10, clf__estimator__gamma=0.001, clf__estimator__kernel=rbf 
[CV] clf__estimator__C=10, clf__estimator__gamma=0.001, clf__estimator__kernel=rbf 
[CV]  clf__estimator__C=10, clf__estimator__gamma=0.001, clf__estimator__kernel=rbf, score=0.0, total= 2.7min
[CV]  clf__estimator__C=10, clf__estimator__gamma=0.001, clf__estimator__kernel=rbf, score=0.0, total= 2.7min


[Parallel(n_jobs=3)]: Done   2 out of   2 | elapsed:  3.8min finished
[Parallel(n_jobs=3)]: Done   2 out of   2 | elapsed:  3.8min remaining:    0.0s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=-1))]
Applying best classifier on test data:
grid_search_tune.best_estimator :  ('clf', OneVsRestClassifier(estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=-1))


### 2.4.4.2 ClassifierChain

In [32]:
chains = [ClassifierChain(SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, shrinking=True,
  tol=0.001, verbose=False), random_state=i)
          for i in range(10)]
i=0
for chain in chains:
    print(i+1)
    i+=1
    chain.fit(x_train_10, y_train_10)

1


ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=0)

2


ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=1)

3


...

ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=7)

9


ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=8)

10


ClassifierChain(base_estimator=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
        cv=None, order=None, random_state=9)

In [33]:
y_pred_chains = np.array([chain.predict(x_test_10) for chain in
                          chains])
predict_svm_ensemble_ClassifierChain_Word2Vec_10 = y_pred_chains.mean(axis=0)



temp = []
org = []
for i in range(len(predict_svm_ensemble_ClassifierChain_Word2Vec_10)):
    for j in range(len(predict_svm_ensemble_ClassifierChain_Word2Vec_10[0])):
        temp.append(int(predict_svm_ensemble_ClassifierChain_Word2Vec_10[i][j]+0.8))
    org.append(temp)
    temp = []
    
predict_svm_ensemble_ClassifierChain_Word2Vec_10 = np.asarray(org)

In [34]:
cc = CC(h=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, shrinking=True,
  tol=0.001, verbose=False))
cc.fit(x_train_10,y_train_10)
save_model(cc,'SVM_Chain_CC_Word2Vec_10.sav','Word2Vec_10')

In [35]:
br = BR(h=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, shrinking=True,
  tol=0.001, verbose=False))
br.fit(x_train_10,y_train_10)
save_model(br,'SVM_Chain_BR_Word2Vec_10.sav','Word2Vec_10')

In [36]:
mcc = MCC(h=SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, shrinking=True,
  tol=0.001, verbose=False))
mcc.fit(x_train_10,y_train_10)
save_model(mcc,'SVM_Chain_MCC_Word2Vec_10.sav','Word2Vec_10')

In [37]:
predict_svm_CC_ClassifierChain_Word2Vec_10=cc.predict(x_test_10)
predict_svm_BR_ClassifierChain_Word2Vec_10=br.predict(x_test_10)
predict_svm_MCC_ClassifierChain_Word2Vec_10=mcc.predict(x_test_10)

##### 2.4.4.3 Scores and visualization


In [38]:
f1_score_svm_OnevsResClassifier_svm_Word2Vec_10 = f1_score(y_test_10,prediction_SVM_OneVsRestClassifier_Word2Vec_10,average='weighted')
f1_score_svm_pure_chain_svm_Word2Vec_10 = f1_score(y_test_10,predict_svm_ensemble_ClassifierChain_Word2Vec_10,average='weighted')
f1_score_svm_CC_svm_Word2Vec_10 = f1_score(y_test_10,predict_svm_CC_ClassifierChain_Word2Vec_10,average='weighted')
f1_score_svm_BR_svm_Word2Vec_10 = f1_score(y_test_10,predict_svm_BR_ClassifierChain_Word2Vec_10,average='weighted')
f1_score_svm_MCC_svm_Word2Vec_10 = f1_score(y_test_10,predict_svm_MCC_ClassifierChain_Word2Vec_10,average='weighted')

print('f1_score_svm_OnevsResClassifier_svm_Word2Vec_10: ',f1_score_svm_OnevsResClassifier_svm_Word2Vec_10)
print('f1_score_svm_pure_chain_svm_Word2Vec_10: ',f1_score_svm_pure_chain_svm_Word2Vec_10)
print('f1_score_svm_CC_ClassifierChain_Word2Vec_10: ',f1_score_svm_CC_svm_Word2Vec_10)
print('f1_score_svm_BR_ClassifierChain_Word2Vec_10: ',f1_score_svm_BR_svm_Word2Vec_10)
print('f1_score_svm_MCC_ClassifierChain_Word2Vec_10: ',f1_score_svm_MCC_svm_Word2Vec_10)

f1_score_svm_OnevsResClassifier_svm_Word2Vec_10:  0.000703726705372
f1_score_svm_pure_chain_svm_Word2Vec_10:  0.0772531980456
f1_score_svm_CC_ClassifierChain_Word2Vec_10:  0.0772531980456
f1_score_svm_BR_ClassifierChain_Word2Vec_10:  0.000703726705372
f1_score_svm_MCC_ClassifierChain_Word2Vec_10:  0.440253558291


In [39]:
Hamming_loss_svm_OnevsResClassifier_svm_Word2Vec_10 = Hamming_loss(y_test_10,prediction_SVM_OneVsRestClassifier_Word2Vec_10)
Hamming_loss_svm_pure_chain_svm_Word2Vec_10 = Hamming_loss(y_test_10,predict_svm_ensemble_ClassifierChain_Word2Vec_10)
Hamming_loss_svm_CC_svm_Word2Vec_10 = Hamming_loss(y_test_10,predict_svm_CC_ClassifierChain_Word2Vec_10)
Hamming_loss_svm_BR_svm_Word2Vec_10 = Hamming_loss(y_test_10,predict_svm_BR_ClassifierChain_Word2Vec_10)
Hamming_loss_svm_MCC_svm_Word2Vec_10 = Hamming_loss(y_test_10,predict_svm_MCC_ClassifierChain_Word2Vec_10)

print('Hamming_loss_svm_OnevsResClassifier_svm_Word2Vec_10: ',Hamming_loss_svm_OnevsResClassifier_svm_Word2Vec_10)
print('Hamming_loss_svm_pure_chain_svm_Word2Vec_10: ',Hamming_loss_svm_pure_chain_svm_Word2Vec_10)
print('Hamming_loss_svm_CC_ClassifierChain_Word2Vec_10: ',Hamming_loss_svm_CC_svm_Word2Vec_10)
print('Hamming_loss_svm_BR_ClassifierChain_Word2Vec_10: ',Hamming_loss_svm_BR_svm_Word2Vec_10)
print('Hamming_loss_svm_MCC_ClassifierChain_Word2Vec_10: ',Hamming_loss_svm_MCC_svm_Word2Vec_10)

Hamming_loss_svm_OnevsResClassifier_svm_Word2Vec_10:  0.0477441077441
Hamming_loss_svm_pure_chain_svm_Word2Vec_10:  0.0466947250281
Hamming_loss_svm_CC_ClassifierChain_Word2Vec_10:  0.0466947250281
Hamming_loss_svm_BR_ClassifierChain_Word2Vec_10:  0.0477441077441
Hamming_loss_svm_MCC_ClassifierChain_Word2Vec_10:  0.0442368125701


In [40]:
Hamming_score_svm_OnevsResClassifier_svm_Word2Vec_10 = Hamming_score(y_test_10,prediction_SVM_OneVsRestClassifier_Word2Vec_10)
Hamming_score_svm_pure_chain_svm_Word2Vec_10 = Hamming_score(y_test_10,predict_svm_ensemble_ClassifierChain_Word2Vec_10)
Hamming_score_svm_CC_svm_Word2Vec_10 = Hamming_score(y_test_10,predict_svm_CC_ClassifierChain_Word2Vec_10)
Hamming_score_svm_BR_svm_Word2Vec_10 = Hamming_score(y_test_10,predict_svm_BR_ClassifierChain_Word2Vec_10)
Hamming_score_svm_MCC_svm_Word2Vec_10 = Hamming_score(y_test_10,predict_svm_MCC_ClassifierChain_Word2Vec_10)

print('Hamming_score_svm_OnevsResClassifier_svm_Word2Vec_10: ',Hamming_score_svm_OnevsResClassifier_svm_Word2Vec_10)
print('Hamming_score_svm_pure_chain_svm_Word2Vec_10: ',Hamming_score_svm_pure_chain_svm_Word2Vec_10)
print('Hamming_score_svm_CC_ClassifierChain_Word2Vec_10: ',Hamming_score_svm_CC_svm_Word2Vec_10)
print('Hamming_score_svm_BR_ClassifierChain_Word2Vec_10: ',Hamming_score_svm_BR_svm_Word2Vec_10)
print('Hamming_score_svm_MCC_ClassifierChain_Word2Vec_10: ',Hamming_score_svm_MCC_svm_Word2Vec_10)

Hamming_score_svm_OnevsResClassifier_svm_Word2Vec_10:  0.952255892256
Hamming_score_svm_pure_chain_svm_Word2Vec_10:  0.953305274972
Hamming_score_svm_CC_ClassifierChain_Word2Vec_10:  0.953305274972
Hamming_score_svm_BR_ClassifierChain_Word2Vec_10:  0.952255892256
Hamming_score_svm_MCC_ClassifierChain_Word2Vec_10:  0.95576318743


In [41]:
Exact_match_svm_OnevsResClassifier_svm_Word2Vec_10 = Exact_match(y_test_10,prediction_SVM_OneVsRestClassifier_Word2Vec_10)
Exact_match_svm_pure_chain_svm_Word2Vec_10 = Exact_match(y_test_10,predict_svm_ensemble_ClassifierChain_Word2Vec_10)
Exact_match_svm_CC_svm_Word2Vec_10 = Exact_match(y_test_10,predict_svm_CC_ClassifierChain_Word2Vec_10)
Exact_match_svm_BR_svm_Word2Vec_10 = Exact_match(y_test_10,predict_svm_BR_ClassifierChain_Word2Vec_10)
Exact_match_svm_MCC_svm_Word2Vec_10 = Exact_match(y_test_10,predict_svm_MCC_ClassifierChain_Word2Vec_10)

print('Exact_match_svm_OnevsResClassifier_svm_Word2Vec_10: ',Exact_match_svm_OnevsResClassifier_svm_Word2Vec_10)
print('Exact_match_svm_pure_chain_svm_Word2Vec_10: ',Exact_match_svm_pure_chain_svm_Word2Vec_10)
print('Exact_match_svm_CC_ClassifierChain_Word2Vec_10: ',Exact_match_svm_CC_svm_Word2Vec_10)
print('Exact_match_svm_BR_ClassifierChain_Word2Vec_10: ',Exact_match_svm_BR_svm_Word2Vec_10)
print('Exact_match_svm_MCC_ClassifierChain_Word2Vec_10: ',Exact_match_svm_MCC_svm_Word2Vec_10)

Exact_match_svm_OnevsResClassifier_svm_Word2Vec_10:  0.000454545454545
Exact_match_svm_pure_chain_svm_Word2Vec_10:  0.0463636363636
Exact_match_svm_CC_ClassifierChain_Word2Vec_10:  0.0463636363636
Exact_match_svm_BR_ClassifierChain_Word2Vec_10:  0.000454545454545
Exact_match_svm_MCC_ClassifierChain_Word2Vec_10:  0.365606060606


### 2.4.5 bp_mll

In [25]:
n = x_train_10.shape[0]
dim_no = x_train_10.shape[1]
class_no = y_train_10.shape[1]

In [29]:
model = Sequential()
model.add(Dense(300, input_dim=dim_no, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dropout(0.6))
model.add(Dense(128, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dropout(0.6))
model.add(Dense(64, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(class_no, activation='sigmoid', kernel_initializer='glorot_uniform'))
model.compile(loss=bp_mll_loss, optimizer='adagrad', metrics=[])


model.fit(x_train_10, y_train_10, epochs=50)

Epoch 1/50
10000/10000 [==============================] - 2s 170us/step - loss: 21.0852
Epoch 2/50
10000/10000 [==============================] - 2s 163us/step - loss: 20.3376
Epoch 3/50
10000/10000 [==============================] - 2s 180us/step - loss: 19.7188
Epoch 4/50
10000/10000 [==============================] - 2s 183us/step - loss: 19.3751
Epoch 5/50
10000/10000 [==============================] - 2s 191us/step - loss: 19.1146
Epoch 6/50
10000/10000 [==============================] - 2s 198us/step - loss: 18.9425
Epoch 7/50
10000/10000 [==============================] - 2s 206us/step - loss: 18.8747
Epoch 8/50
10000/10000 [==============================] - 2s 204us/step - loss: 18.7814
Epoch 9/50
10000/10000 [==============================] - 2s 203us/step - loss: 18.7363
Epoch 10/50
10000/10000 [==============================] - 2s 202us/step - loss: 18.6884
Epoch 11/50
10000/10000 [==============================] - 2s 199us/step - loss: 18.6517
Epoch 12/50
10000/10000 [=====

In [31]:
bpmll_Word2Vec = model.predict(x_test_10)


res = []
tmp = []

for j in range(bpmll_Word2Vec.shape[0]):
    for i in range(bpmll_Word2Vec.shape[1]):
        tmp.append(int(bpmll_Word2Vec[j][i]+0.01))
    res.append(tmp)
    tmp = []

res = pd.DataFrame(res)
res = res.as_matrix()


print('BP-MLL F1-Score: ',f1_score(y_test_10, res,average='weighted'))
print('BP-MLL Hamming_loss: ',Hamming_loss(y_test_10, res))
print('BP-MLL Hamming_score: ',Hamming_score(y_test_10, res))
print('BP-MLL Exact_match: ',Exact_match(y_test_10, res))









BP-MLL F1-Score:  0.42315564035541475

BP-MLL Hamming_loss:  0.10385185185185175

BP-MLL Hamming_score:  0.8961481481481482

BP-MLL Exact_match:  0.09
